<a href="https://colab.research.google.com/github/BiniyamAschalew/BiniyamAschalew/blob/main/2organized343.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [161]:
from google.colab import drive
drive.mount('content')
input_dir = "content/MyDrive/IE343/"

Drive already mounted at content; to attempt to forcibly remount, call drive.mount("content", force_remount=True).


In [162]:
from google.colab import files

In [163]:
# !pip install optuna
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [164]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import  HistGradientBoostingRegressor,RandomForestRegressor
from sklearn.preprocessing import  LabelEncoder, StandardScaler
from sklearn.model_selection import TimeSeriesSplit, KFold, train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import re
import geopandas as gpd
# import optuna
# from optuna.integration import XGBoostPruningCallback

In [165]:
from scipy.spatial import distance
from sklearn.neighbors import BallTree

In [166]:
train_data = pd.read_csv(input_dir + "train.csv")
test_data = pd.read_csv(input_dir + "test.csv")
seoul_subway0 = pd.read_csv(input_dir + "Seoul_subway_stations.csv")
busan_subway0 = gpd.read_file(input_dir + 'Shape_files/Busan_subway.shp').to_crs(epsg=4326)

In [167]:
seoul_subway = seoul_subway0[['lat', 'lng']]
seoul_subway.rename(columns={'lng':'long'}, inplace=True)
seoul_subway

<ipython-input-167-67fb2af9d7c2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seoul_subway.rename(columns={'lng':'long'}, inplace=True)


,lat,long
0,37.948100,127.061034
1,37.927878,127.054790
2,37.913702,127.057277
3,37.901885,127.056482
4,37.892334,127.055716
...,...,...
704,37.317185,127.404760
705,37.265579,127.442260
706,37.260192,127.490277
707,37.295309,127.570938


In [168]:
# preprocessing

busan_subway = busan_subway0['geometry'].apply(lambda x: x.coords[0][::-1]).tolist()
busan_subway = pd.DataFrame(busan_subway, columns=['lat', 'long'])
busan_subway

,lat,long
0,35.169101,128.950703
1,35.163607,128.956957
2,35.160308,128.979736
3,35.179178,128.956271
4,35.193403,128.965648
...,...,...
114,35.096974,128.985480
115,35.100812,128.975726
116,35.103371,128.969096
117,35.092484,128.962904


In [169]:
subway = pd.concat([busan_subway, seoul_subway], ignore_index=True)
subway

,lat,long
0,35.169101,128.950703
1,35.163607,128.956957
2,35.160308,128.979736
3,35.179178,128.956271
4,35.193403,128.965648
...,...,...
823,37.317185,127.404760
824,37.265579,127.442260
825,37.260192,127.490277
826,37.295309,127.570938


In [170]:
tr = train_data.copy()
tst = test_data.copy()

In [171]:
# change the columns with object dtype to numbers by encoding them
le = LabelEncoder()
label_columns = list(tr.select_dtypes('object'))
for col in label_columns:
  le.fit(tr[col])
  tr[col] = le.transform(tr[col])
  tst[col] = le.transform(tst[col])

In [172]:
min_year = tr['transaction_year'].min()
tr['cpi'] = (tr['transaction_year'] - min_year) * 12 + tr['transaction_month']
tr['age'] = tr['transaction_year'] - tr['built_year']

tst['cpi'] = (tst['transaction_year'] - min_year) * 12 + tst['transaction_month']
tst['age'] = tst['transaction_year'] - tst['built_year']

tr.head()
tr.dtypes

index                  int64
apartment_id           int64
city                   int64
dong                   int64
house_area           float64
built_year             int64
floor                  int64
lat                  float64
long                 float64
transaction_year       int64
transaction_month      int64
transaction_day        int64
PRICE                float64
cpi                    int64
age                    int64
dtype: object

In [173]:
# see the effect at the end

# scale lat and long
# scaler_lat = StandardScaler()
# scaler_long = StandardScaler()

# # tr.loc[tr['dong'] != 113,'lat'] = scaler_lat.fit_transform(tr[tr['dong'] != 113]['lat'])
# scaler_lat.fit(tr[tr['dong'] != 113][['lat']])
# tr.loc[tr['dong'] != 113, 'lat'] = scaler_lat.transform(tr[tr['dong'] != 113][['lat']])
# tst.loc[tst['dong'] != 113, 'lat'] = scaler_lat.transform(tst[tst['dong'] != 113][['lat']])

# scaler_long.fit(tr[tr['dong'] != 113][['long']])
# tr.loc[tr['dong'] != 113, 'long'] = scaler_long.transform(tr[tr['dong'] != 113][['long']])
# tst.loc[tst['dong'] != 113, 'long'] = scaler_long.transform(tst[tst['dong'] != 113][['long']])

# tr.loc[:, 'lat'].mean()

In [174]:
# Taking care of missing values
# the only missing values exist in busan dong 113
# solution lets assign its values with normal distribution with mean -> average of all lat and long in busan, sd of sd of the others
busan_dong_train = tr[tr['city'] == 0].loc[:, ["dong", "lat", "long"]]
busan_dong_test = tst[tst['city'] == 0].loc[:, ["dong", "lat", "long"]]
busan_dong = pd.concat([busan_dong_train, busan_dong_test])

mean_lat1, mean_long1 = busan_dong[busan_dong['dong'] == 115].mean()[1:]
mean_lat2, mean_long2 = busan_dong[busan_dong['dong'] == 114].mean()[1:]
mean_lat = (mean_lat1 + mean_lat2)/2
mean_long = (mean_long1 + mean_long2)/2
mean_lat, mean_long

std_lat1, std_long1 = busan_dong[busan_dong['dong'] == 115].std()[1:]
std_lat2, std_long2 = busan_dong[busan_dong['dong'] == 114].std()[1:]
std_lat = (std_lat1 + std_lat2)/2
std_long = (std_long1 + std_long2)/2
std_lat, std_long

na_train = 89
na_test = 13


# busan_dong.head()


In [175]:
# now let's scale lat and long to have standard normal dist


In [176]:

tst.loc[tst['dong'] == 113,'lat'] = np.random.randn(na_test) * std_lat + mean_lat
tst.loc[tst['dong'] == 113,'long'] = np.random.randn(na_test) * std_long + mean_long

tr.loc[tr['dong'] == 113,'lat'] = np.random.randn(na_train) * std_lat + mean_lat
tr.loc[tr['dong'] == 113,'long'] = np.random.randn(na_train) * std_long + mean_long

In [177]:
# since the apartment id is random hash name of apartments, lets converted it to also represent the average price of that apartment frenchise

tr['apartment_rank'] = np.zeros(len(tr))
tst['apartment_rank'] = np.zeros(len(tst))


apartment_price = tr.groupby('apartment_id')['PRICE'].agg('mean').sort_values(ascending=False)
apartment_price # I want to sort them and put the sorted
for idx, apartment in enumerate(list(apartment_price.index)):
  tr.loc[tr['apartment_id'] == apartment, 'apartment_rank' ] = idx
  tst.loc[tst['apartment_id'] == apartment, 'apartment_rank' ] = idx

# sns.regplot(x=tr['apartment_rank'], y = tr['PRICE'])

In [178]:
# add distance to subway stations as a feature
homes_rad_tr = np.radians(tr[['lat', 'long']])
homes_rad_tst = np.radians(tst[['lat', 'long']])
subway_rad = np.radians(subway)

tree = BallTree(subway_rad, metric='haversine')

distance_tr, indices_tr = tree.query(homes_rad_tr, k=3)
distance_tst, indices_tst = tree.query(homes_rad_tst, k = 3)

distance_tr *= 6371
distance_tst *= 6371

distance_tr3 = distance_tr.sum(axis=1)
distance_tst3 = distance_tst.sum(axis=1)

In [179]:
distance_tr1 = distance_tr.min(axis=1)
distance_tst1 = distance_tst.min(axis=1)



In [180]:
# add the sum of the distance to the nearest 3 subways and the min distance to a subway to the featuer set
tr['dist1'] = distance_tr1
tr['dist3'] = distance_tr3

tst['dist1'] = distance_tst1
tst['dist3'] = distance_tst3

tr

,index,apartment_id,city,dong,house_area,built_year,floor,lat,long,transaction_year,transaction_month,transaction_day,PRICE,cpi,age,apartment_rank,dist1,dist3
0,0,0,0,197,125.865988,1993,5,35.149929,129.006071,2021,7,0,229250.80,43,28,3180.0,0.267518,2.206177
1,1,0,0,197,101.647190,1993,12,35.149929,129.006071,2021,10,1,215320.00,46,28,3180.0,0.267518,2.206177
2,2,0,0,197,91.511175,1993,6,35.149929,129.006071,2020,3,5,161740.00,27,27,3180.0,0.267518,2.206177
3,3,0,0,197,101.647190,1993,13,35.149929,129.006071,2020,5,0,199781.80,29,27,3180.0,0.267518,2.206177
4,4,0,0,197,101.647190,1993,4,35.149929,129.006071,2022,6,4,219606.40,54,29,3180.0,0.267518,2.206177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329685,329685,4419,1,37,101.431912,2014,4,37.452039,127.070842,2022,5,5,885070.00,53,8,396.0,1.443422,8.650589
329686,329686,4419,1,37,101.431912,2014,14,37.452039,127.070842,2021,10,1,826132.00,46,7,396.0,1.443422,8.650589
329687,329687,4419,1,37,71.687641,2014,2,37.452039,127.070842,2022,11,4,697540.00,59,8,396.0,1.443422,8.650589
329688,329688,4419,1,37,137.192013,2014,18,37.452039,127.070842,2020,9,4,870656.98,33,6,396.0,1.443422,8.650589


In [196]:
x_train2 = tr.copy()
x_test2 = tst.copy()

finx_train = x_train2.drop(columns = 'PRICE')
finy_train = x_train2['PRICE']
x_test2

,index,apartment_id,city,dong,house_area,built_year,floor,lat,long,transaction_year,transaction_month,transaction_day,cpi,age,apartment_rank,dist1,dist3
0,329690,0,0,197,101.647190,1993,3,35.149929,129.006071,2023,1,5,61,30,3180.0,0.267518,2.206177
1,329691,0,0,197,91.511175,1993,12,35.149929,129.006071,2023,2,1,62,30,3180.0,0.267518,2.206177
2,329692,0,0,197,125.865988,1993,2,35.149929,129.006071,2023,2,0,62,30,3180.0,0.267518,2.206177
3,329693,0,0,197,101.647190,1993,8,35.149929,129.006071,2023,2,2,62,30,3180.0,0.267518,2.206177
4,329694,0,0,197,101.647190,1993,13,35.149929,129.006071,2023,3,5,63,30,3180.0,0.267518,2.206177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85092,414782,4419,1,37,100.821957,2014,8,37.452039,127.070842,2023,10,5,70,9,396.0,1.443422,8.650589
85093,414783,4419,1,37,101.431912,2014,11,37.452039,127.070842,2023,10,5,70,9,396.0,1.443422,8.650589
85094,414784,4419,1,37,121.201627,2014,12,37.452039,127.070842,2023,11,1,71,9,396.0,1.443422,8.650589
85095,414785,4419,1,37,137.192013,2014,3,37.452039,127.070842,2023,11,4,71,9,396.0,1.443422,8.650589


In [200]:
x_test2.columns

Index(['index', 'apartment_id', 'city', 'dong', 'house_area', 'built_year',
       'floor', 'lat', 'long', 'transaction_year', 'transaction_month',
       'transaction_day', 'cpi', 'age', 'apartment_rank', 'dist1', 'dist3'],
      dtype='object')

In [199]:
model_lgb8 = lgb.LGBMRegressor( n_estimators = 10000)
model_lgb8.fit(finx_train, finy_train)

y_pred8 = model_lgb8.predict(x_test2)

output = pd.concat([x_test2['index'], pd.DataFrame(y_pred8, columns = ["PRICE"])], axis = 1)

output.to_csv('subm11.csv', index=False)  
files.download('subm11.csv')
# subm10 with n_est -= 20 k and subway data returned 48k
# and with 10k I got 47.6k , it increased with the subway data


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [203]:
from sklearn.decomposition import PCA, FastICA

x_test3 = x_test2.copy()
x_train3 = x_train2.copy()

# I am going to add the number of parks and children center


,index,apartment_id,city,dong,house_area,built_year,floor,lat,long,transaction_year,transaction_month,transaction_day,cpi,age,apartment_rank,dist1,dist3
0,329690,0,0,197,101.647190,1993,3,35.149929,129.006071,2023,1,5,61,30,3180.0,0.267518,2.206177
1,329691,0,0,197,91.511175,1993,12,35.149929,129.006071,2023,2,1,62,30,3180.0,0.267518,2.206177
2,329692,0,0,197,125.865988,1993,2,35.149929,129.006071,2023,2,0,62,30,3180.0,0.267518,2.206177
3,329693,0,0,197,101.647190,1993,8,35.149929,129.006071,2023,2,2,62,30,3180.0,0.267518,2.206177
4,329694,0,0,197,101.647190,1993,13,35.149929,129.006071,2023,3,5,63,30,3180.0,0.267518,2.206177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85092,414782,4419,1,37,100.821957,2014,8,37.452039,127.070842,2023,10,5,70,9,396.0,1.443422,8.650589
85093,414783,4419,1,37,101.431912,2014,11,37.452039,127.070842,2023,10,5,70,9,396.0,1.443422,8.650589
85094,414784,4419,1,37,121.201627,2014,12,37.452039,127.070842,2023,11,1,71,9,396.0,1.443422,8.650589
85095,414785,4419,1,37,137.192013,2014,3,37.452039,127.070842,2023,11,4,71,9,396.0,1.443422,8.650589


In [182]:
# model_lgb = lgb.LGBMRegressor(objective='regression', max_depth=7, learning_rate=0.0007953, n_estimators = 8829, min_child_samples = 85, subsample=0.5223234814923825)
# model_lgb.fit(finx_train, finy_train)



In [183]:
# # new one
# model_lgb2 = lgb.LGBMRegressor( max_depth=7, learning_rate=0.001, n_estimators = 10000, min_child_samples = 82, subsample=0.5223234814923825)
# model_lgb2.fit(finx_train, finy_train)

# y_pred2 = model_lgb2.predict(x_test2)

# output = pd.concat([x_test2['index'], pd.DataFrame(y_pred2, columns = ["PRICE"])], axis = 1)

# output.to_csv('subm6.csv', index=False)
# files.download('subm6.csv')

In [184]:
# model_lgb3 = lgb.LGBMRegressor( n_estimators = 10000)
# model_lgb3.fit(finx_train, finy_train)

# y_pred3 = model_lgb3.predict(x_test2)

# output = pd.concat([x_test2['index'], pd.DataFrame(y_pred3, columns = ["PRICE"])], axis = 1)

# output.to_csv('subm7.csv', index=False)  
# files.download('subm7.csv')

In [185]:
# model_lgb4 = lgb.LGBMRegressor( )
# model_lgb4.fit(finx_train, finy_train)

In [186]:
# model_lgb4 = lgb.LGBMRegressor()
# model_lgb4.fit(finx_train, finy_train)

# y_pred3 = model_lgb4.predict(x_test2)

# output = pd.concat([x_test2['index'], pd.DataFrame(y_pred3, columns = ["PRICE"])], axis = 1)

# output.to_csv('subm8.csv', index=False)
# files.download('subm8.csv')

In [187]:
# model_lgb5 = lgb.LGBMRegressor(n_estimators=15000)
# model_lgb5.fit(finx_train, finy_train)

# y_pred4 = model_lgb5.predict(x_test2)

# output = pd.concat([x_test2['index'], pd.DataFrame(y_pred4, columns = ["PRICE"])], axis = 1)

# output.to_csv('subm9.csv', index=False)
# files.download('subm9.csv')

In [188]:
# finx_train # normalizing lat and long had almost 0 effect

In [189]:
# model_lgb

# y_pred = model_lgb.predict(x_test2)

# output = pd.concat([tst['index'], pd.DataFrame(y_pred, columns = ["PRICE"])], axis = 1)

In [190]:
# {'max_depth': 7, 'learning_rate': 0.0007953098131062764, 'n_estimators': 8829, 'min_child_samples': 85, 'subsample': 0.5223234814923825} trial 14  value: 44711.82480833971.
# {'max_depth': 11, 'learning_rate': 0.005252427629146827, 'n_estimators': 8015, 'min_child_samples': 54, 'subsample': 0.8422371309477078} trial 2 with value: 47773.23330978951.
# {'max_depth': 7, 'learning_rate': 0.001023558187447401, 'n_estimators': 9524, 'min_child_samples': 82, 'subsample': 0.4785940319002228}. Best is trial 21 with value: 44231.935269501104

In [191]:
# # subm4 was with the second hyperparameter above, it got 61k 
# model_lgb

In [192]:
# output.to_csv('subm5.csv', index=False)
# files.download('subm5.csv')

In [193]:
# lets try it without normalizing the lat and long
# subm 5 is without normalizing lat and long

In [194]:
x_test2

,index,apartment_id,city,dong,house_area,built_year,floor,lat,long,transaction_year,transaction_month,transaction_day,cpi,age,apartment_rank,dist1,dist3
0,329690,0,0,197,101.647190,1993,3,35.149929,129.006071,2023,1,5,61,30,3180.0,0.267518,2.206177
1,329691,0,0,197,91.511175,1993,12,35.149929,129.006071,2023,2,1,62,30,3180.0,0.267518,2.206177
2,329692,0,0,197,125.865988,1993,2,35.149929,129.006071,2023,2,0,62,30,3180.0,0.267518,2.206177
3,329693,0,0,197,101.647190,1993,8,35.149929,129.006071,2023,2,2,62,30,3180.0,0.267518,2.206177
4,329694,0,0,197,101.647190,1993,13,35.149929,129.006071,2023,3,5,63,30,3180.0,0.267518,2.206177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85092,414782,4419,1,37,100.821957,2014,8,37.452039,127.070842,2023,10,5,70,9,396.0,1.443422,8.650589
85093,414783,4419,1,37,101.431912,2014,11,37.452039,127.070842,2023,10,5,70,9,396.0,1.443422,8.650589
85094,414784,4419,1,37,121.201627,2014,12,37.452039,127.070842,2023,11,1,71,9,396.0,1.443422,8.650589
85095,414785,4419,1,37,137.192013,2014,3,37.452039,127.070842,2023,11,4,71,9,396.0,1.443422,8.650589
